In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [2]:
cv=LeaveOneOut()

In [3]:
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [4]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    param1=["best","random"]
    score=0
    best_param=param1[0]
    for i in param1:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=i)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        if ACC>score:
            score=ACC
            best_param=i
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param1,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    result_final = pd.DataFrame(results)
    return best_param,results

In [5]:
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
   # print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    score=0
    best_param=param2[0]
    #param1=["best","random"]
    for i in param2:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=i)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        if ACC>score:
            score=ACC
            best_param=i
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param2,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    result_final = pd.DataFrame(results)
  #  return grid_search.best_params_["splitter"],results
    return best_param,results

In [6]:
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,50))  ## 设置max_features范围
  #  param2=np.array(range(3,10,1))
   # print(datetime.datetime.now())
    TN_a=[]
    FN_a=[]
    FP_a=[]
    TP_a=[]
    accuracy_a=[]
    pre_a=[]
    rec_a=[]
    f1_a=[]
    auroc_a=[]
    #param1=["best","random"]
    for j in param3:
        Predict_Y=[]    #存放预测值
        Prob_Y=[]       #存放预测概率
        Really_Y=y
        for train_index,test_index in cv.split(X):
            estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=j)
            estimator.fit(X[train_index],y[train_index])
            Predict_Y.append(estimator.predict(X[test_index])[0])
            Prob_Y.append(estimator.predict_proba(X[test_index]))
        Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
        Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
        Prob_Y=pd.DataFrame(Prob_Y)
        Prob_Y.columns=["0","1"]
        confusion_matrix(Really_Y,Predict_Y)
        TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
        FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
        FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
        TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
        ACC=accuracy_score(Really_Y,Predict_Y)
        Precision=precision_score(Really_Y,Predict_Y)
        Recall=recall_score(Really_Y,Predict_Y)
        F1_score=f1_score(Really_Y,Predict_Y)
        AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
        TN_a.append(TN)
        FN_a.append(FN)
        FP_a.append(FP)
        TP_a.append(TP)
        accuracy_a.append(ACC)
        pre_a.append(Precision)
        rec_a.append(Recall)
        f1_a.append(F1_score)
        auroc_a.append(AUC)  
    results=np.array([param3,TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a])
    return results

In [7]:
file_path = 'E:\datatest\PseDPC\d5\d51'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
511.csv
**************************************************
2022-02-24 16:35:22.857784
2022-02-24 16:35:26.268658
2022-02-24 16:35:42.363596
##################################################
512.csv
**************************************************
2022-02-24 16:35:48.603901
2022-02-24 16:35:51.743501
2022-02-24 16:36:07.036585
##################################################
513.csv
**************************************************
2022-02-24 16:36:14.852672
2022-02-24 16:36:18.365275
2022-02-24 16:36:36.094838
##################################################
514.csv
**************************************************
2022-02-24 16:36:43.785263
2022-02-24 16:36:46.744346
2022-02-24 16:37:01.233580
##################################################
515.csv
**************************************************
2022-02-24 16:37:08.417360
2022-02-24 16:37:11.501109
2022-02-24 16:37:16.583512
#############################################

In [8]:
file_path = 'E:\datatest\PseDPC\d5\d52'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
521.csv
**************************************************
2022-02-24 16:38:59.665715
2022-02-24 16:39:02.575929
2022-02-24 16:39:16.758983
##################################################
522.csv
**************************************************
2022-02-24 16:39:22.539517
2022-02-24 16:39:25.497602
2022-02-24 16:39:39.868154
##################################################
523.csv
**************************************************
2022-02-24 16:39:45.641707
2022-02-24 16:39:48.707505
2022-02-24 16:40:03.864952
##################################################
524.csv
**************************************************
2022-02-24 16:40:10.019485
2022-02-24 16:40:13.002503
2022-02-24 16:40:28.033289
##################################################
525.csv
**************************************************
2022-02-24 16:40:34.181839
2022-02-24 16:40:37.182810
2022-02-24 16:40:52.276426
#############################################

In [9]:
file_path = 'E:\datatest\PseDPC\d5\d53'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
531.csv
**************************************************
2022-02-24 16:42:40.891827
2022-02-24 16:42:43.811016
2022-02-24 16:42:57.787622
##################################################
532.csv
**************************************************
2022-02-24 16:43:04.664224
2022-02-24 16:43:07.460742
2022-02-24 16:43:21.365539
##################################################
533.csv
**************************************************
2022-02-24 16:43:28.012755
2022-02-24 16:43:30.938928
2022-02-24 16:43:45.408213
##################################################
534.csv
**************************************************
2022-02-24 16:43:51.340342
2022-02-24 16:43:54.430075
2022-02-24 16:44:09.357139
##################################################
535.csv
**************************************************
2022-02-24 16:44:16.111067
2022-02-24 16:44:19.345414
2022-02-24 16:44:35.380512
#############################################

In [10]:
file_path = 'E:\datatest\PseDPC\d5\d54'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
541.csv
**************************************************
2022-02-24 16:45:54.169715
2022-02-24 16:45:57.309313
2022-02-24 16:46:12.693153
##################################################
542.csv
**************************************************
2022-02-24 16:46:19.681456
2022-02-24 16:46:22.735286
2022-02-24 16:46:37.656364
##################################################
543.csv
**************************************************
2022-02-24 16:46:44.768336
2022-02-24 16:46:48.075489
2022-02-24 16:47:04.745886
##################################################
544.csv
**************************************************
2022-02-24 16:47:12.940961
2022-02-24 16:47:16.234149
2022-02-24 16:47:33.034202
##################################################
545.csv
**************************************************
2022-02-24 16:47:40.425441
2022-02-24 16:47:43.719612
2022-02-24 16:48:00.646326
#############################################

In [11]:
file_path = 'E:\datatest\PseDPC\d5\d55'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
551.csv
**************************************************
2022-02-24 16:49:37.425394
2022-02-24 16:49:40.748503
2022-02-24 16:49:57.083797
##################################################
552.csv
**************************************************
2022-02-24 16:50:04.342377
2022-02-24 16:50:07.609637
2022-02-24 16:50:23.854174
##################################################
553.csv
**************************************************
2022-02-24 16:50:31.017009
2022-02-24 16:50:34.233501
2022-02-24 16:50:39.323784
##################################################
554.csv
**************************************************
2022-02-24 16:50:42.381603
2022-02-24 16:50:45.722664
2022-02-24 16:51:02.799974
##################################################
555.csv
**************************************************
2022-02-24 16:51:08.934561
2022-02-24 16:51:12.338454
2022-02-24 16:51:29.420751
#############################################

In [12]:
file_path = 'E:\datatest\PseDPC\d5\d56'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
561.csv
**************************************************
2022-02-24 16:53:21.587650
2022-02-24 16:53:24.815016
2022-02-24 16:53:40.780299
##################################################
562.csv
**************************************************
2022-02-24 16:53:47.892270
2022-02-24 16:53:51.307134
2022-02-24 16:54:08.302663
##################################################
563.csv
**************************************************
2022-02-24 16:54:14.481132
2022-02-24 16:54:17.728444
2022-02-24 16:54:33.684753
##################################################
564.csv
**************************************************
2022-02-24 16:54:39.876188
2022-02-24 16:54:43.292049
2022-02-24 16:55:00.390302
##################################################
565.csv
**************************************************
2022-02-24 16:55:08.656187
2022-02-24 16:55:12.210677
2022-02-24 16:55:29.354808
#############################################

In [13]:
file_path = 'E:\datatest\PseDPC\d5\d57'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
571.csv
**************************************************
2022-02-24 16:57:29.287958
2022-02-24 16:57:32.238065
2022-02-24 16:57:46.486943
##################################################
572.csv
**************************************************
2022-02-24 16:57:52.413087
2022-02-24 16:57:55.930676
2022-02-24 16:58:13.745014
##################################################
573.csv
**************************************************
2022-02-24 16:58:22.245280
2022-02-24 16:58:25.677090
2022-02-24 16:58:42.781327
##################################################
574.csv
**************************************************
2022-02-24 16:58:50.961456
2022-02-24 16:58:54.434150
2022-02-24 16:59:11.572297
##################################################
575.csv
**************************************************
2022-02-24 16:59:19.886054
2022-02-24 16:59:23.365744
2022-02-24 16:59:40.413134
#############################################

In [14]:
file_path = 'E:\datatest\PseDPC\d5\d58'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
581.csv
**************************************************
2022-02-24 17:01:33.374905
2022-02-24 17:01:36.547417
2022-02-24 17:01:51.918292
##################################################
582.csv
**************************************************
2022-02-24 17:01:58.004010
2022-02-24 17:02:01.092746
2022-02-24 17:02:16.373861
##################################################
583.csv
**************************************************
2022-02-24 17:02:22.510443
2022-02-24 17:02:25.629099
2022-02-24 17:02:40.967063
##################################################
584.csv
**************************************************
2022-02-24 17:02:47.089682
2022-02-24 17:02:50.265186
2022-02-24 17:03:05.729810
##################################################
585.csv
**************************************************
2022-02-24 17:03:12.386999
2022-02-24 17:03:16.080119
2022-02-24 17:03:33.970254
#############################################

In [15]:
file_path = 'E:\datatest\PseDPC\d5\d59'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
591.csv
**************************************************
2022-02-24 17:05:29.923022
2022-02-24 17:05:33.088552
2022-02-24 17:05:48.460426
##################################################
592.csv
**************************************************
2022-02-24 17:05:55.459699
2022-02-24 17:05:58.903484
2022-02-24 17:06:04.070661
##################################################
593.csv
**************************************************
2022-02-24 17:06:06.366517
2022-02-24 17:06:09.904053
2022-02-24 17:06:28.218054
##################################################
594.csv
**************************************************
2022-02-24 17:06:34.515206
2022-02-24 17:06:38.040773
2022-02-24 17:06:55.306578
##################################################
595.csv
**************************************************
2022-02-24 17:07:01.405261
2022-02-24 17:07:04.879965
2022-02-24 17:07:22.183670
#############################################

In [16]:
file_path = 'E:\datatest\PseDPC\d5\d510'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5101.csv
**************************************************
2022-02-24 17:09:21.907350
2022-02-24 17:09:25.686239
2022-02-24 17:09:44.612602
##################################################
5102.csv
**************************************************
2022-02-24 17:09:51.329640
2022-02-24 17:09:55.019757
2022-02-24 17:10:13.805549
##################################################
5103.csv
**************************************************
2022-02-24 17:10:20.220384
2022-02-24 17:10:23.835712
2022-02-24 17:10:41.836553
##################################################
5104.csv
**************************************************
2022-02-24 17:10:48.086828
2022-02-24 17:10:51.751025
2022-02-24 17:10:57.358023
##################################################
5105.csv
**************************************************
2022-02-24 17:11:00.561452
2022-02-24 17:11:04.424118
2022-02-24 17:11:22.798956
########################################

In [17]:
file_path = 'E:\datatest\PseDPC\d5\d511'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5111.csv
**************************************************
2022-02-24 17:13:21.040601
2022-02-24 17:13:24.551209
2022-02-24 17:13:42.075322
##################################################
5112.csv
**************************************************
2022-02-24 17:13:49.261097
2022-02-24 17:13:52.920307
2022-02-24 17:14:11.911498
##################################################
5113.csv
**************************************************
2022-02-24 17:14:20.104096
2022-02-24 17:14:24.499336
2022-02-24 17:14:43.864525
##################################################
5114.csv
**************************************************
2022-02-24 17:14:52.657001
2022-02-24 17:14:56.617405
2022-02-24 17:15:16.331663
##################################################
5115.csv
**************************************************
2022-02-24 17:15:24.565629
2022-02-24 17:15:27.853831
2022-02-24 17:15:45.733996
########################################

In [18]:
file_path = 'E:\datatest\PseDPC\d5\d512'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5121.csv
**************************************************
2022-02-24 17:17:41.501258
2022-02-24 17:17:44.997903
2022-02-24 17:17:50.081302
##################################################
5122.csv
**************************************************
2022-02-24 17:17:53.010465
2022-02-24 17:17:56.504118
2022-02-24 17:18:01.506734
##################################################
5123.csv
**************************************************
2022-02-24 17:18:04.370073
2022-02-24 17:18:08.010333
2022-02-24 17:18:12.972058
##################################################
5124.csv
**************************************************
2022-02-24 17:18:16.093707
2022-02-24 17:18:19.997263
2022-02-24 17:18:25.203333
##################################################
5125.csv
**************************************************
2022-02-24 17:18:28.275115
2022-02-24 17:18:31.883461
2022-02-24 17:18:36.929959
########################################

In [19]:
file_path = 'E:\datatest\PseDPC\d5\d513'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5131.csv
**************************************************
2022-02-24 17:19:24.081805
2022-02-24 17:19:27.764951
2022-02-24 17:19:45.973234
##################################################
5132.csv
**************************************************
2022-02-24 17:19:54.220169
2022-02-24 17:19:58.093806
2022-02-24 17:20:17.243571
##################################################
5133.csv
**************************************************
2022-02-24 17:20:23.534739
2022-02-24 17:20:27.387431
2022-02-24 17:20:46.402556
##################################################
5134.csv
**************************************************
2022-02-24 17:20:52.700705
2022-02-24 17:20:56.564368
2022-02-24 17:21:15.428896
##################################################
5135.csv
**************************************************
2022-02-24 17:21:24.566449
2022-02-24 17:21:28.174795
2022-02-24 17:21:47.779343
########################################

In [20]:
file_path = 'E:\datatest\PseDPC\d5\d514'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5141.csv
**************************************************
2022-02-24 17:24:03.983507
2022-02-24 17:24:07.962861
2022-02-24 17:24:28.342335
##################################################
5142.csv
**************************************************
2022-02-24 17:24:35.981896
2022-02-24 17:24:40.517760
2022-02-24 17:25:03.021552
##################################################
5143.csv
**************************************************
2022-02-24 17:25:10.414771
2022-02-24 17:25:14.882816
2022-02-24 17:25:36.195795
##################################################
5144.csv
**************************************************
2022-02-24 17:25:43.062425
2022-02-24 17:25:47.008863
2022-02-24 17:26:08.721793
##################################################
5145.csv
**************************************************
2022-02-24 17:26:16.042207
2022-02-24 17:26:20.288845
2022-02-24 17:26:40.640394
########################################

In [21]:
file_path = 'E:\datatest\PseDPC\d5\d515'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5151.csv
**************************************************
2022-02-24 17:29:02.303376
2022-02-24 17:29:06.764441
2022-02-24 17:29:29.171491
##################################################
5152.csv
**************************************************
2022-02-24 17:29:37.886174
2022-02-24 17:29:41.846578
2022-02-24 17:30:01.568811
##################################################
5153.csv
**************************************************
2022-02-24 17:30:11.299775
2022-02-24 17:30:15.483582
2022-02-24 17:30:21.017775
##################################################
5154.csv
**************************************************
2022-02-24 17:30:25.419997
2022-02-24 17:30:31.672270
2022-02-24 17:30:53.717289
##################################################
5155.csv
**************************************************
2022-02-24 17:31:02.908697
2022-02-24 17:31:07.059592
2022-02-24 17:31:12.594781
########################################

In [22]:
file_path = 'E:\datatest\PseDPC\d5\d516'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5161.csv
**************************************************
2022-02-24 17:32:25.794935
2022-02-24 17:32:29.644636
2022-02-24 17:32:48.511158
##################################################
5162.csv
**************************************************
2022-02-24 17:32:57.181958
2022-02-24 17:33:01.332853
2022-02-24 17:33:21.225630
##################################################
5163.csv
**************************************************
2022-02-24 17:33:28.620844
2022-02-24 17:33:34.366474
2022-02-24 17:33:54.429794
##################################################
5164.csv
**************************************************
2022-02-24 17:34:01.639503
2022-02-24 17:34:06.058680
2022-02-24 17:34:28.079763
##################################################
5165.csv
**************************************************
2022-02-24 17:34:35.086017
2022-02-24 17:34:38.642502
2022-02-24 17:35:00.596764
########################################

In [23]:
file_path = 'E:\datatest\PseDPC\d5\d517'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5171.csv
**************************************************
2022-02-24 17:37:12.108416
2022-02-24 17:37:15.768623
2022-02-24 17:37:35.233549
##################################################
5172.csv
**************************************************
2022-02-24 17:37:42.835207
2022-02-24 17:37:47.082842
2022-02-24 17:38:08.901466
##################################################
5173.csv
**************************************************
2022-02-24 17:38:16.156058
2022-02-24 17:38:20.335874
2022-02-24 17:38:40.835028
##################################################
5174.csv
**************************************************
2022-02-24 17:38:47.841284
2022-02-24 17:38:51.747831
2022-02-24 17:39:12.556160
##################################################
5175.csv
**************************************************
2022-02-24 17:39:19.799779
2022-02-24 17:39:24.046417
2022-02-24 17:39:44.550557
########################################

In [24]:
file_path = 'E:\datatest\PseDPC\d5\d518'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5181.csv
**************************************************
2022-02-24 17:41:59.682014
2022-02-24 17:42:03.336237
2022-02-24 17:42:22.543847
##################################################
5182.csv
**************************************************
2022-02-24 17:42:29.409478
2022-02-24 17:42:33.322010
2022-02-24 17:42:54.149290
##################################################
5183.csv
**************************************************
2022-02-24 17:43:03.229991
2022-02-24 17:43:07.263202
2022-02-24 17:43:27.874056
##################################################
5184.csv
**************************************************
2022-02-24 17:43:36.007296
2022-02-24 17:43:39.893897
2022-02-24 17:43:59.584217
##################################################
5185.csv
**************************************************
2022-02-24 17:44:06.947516
2022-02-24 17:44:10.864037
2022-02-24 17:44:31.855873
########################################

In [25]:
file_path = 'E:\datatest\PseDPC\d5\d519'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5191.csv
**************************************************
2022-02-24 17:46:54.150166
2022-02-24 17:46:57.699670
2022-02-24 17:47:16.349771
##################################################
5192.csv
**************************************************
2022-02-24 17:47:25.278882
2022-02-24 17:47:29.130577
2022-02-24 17:47:48.962516
##################################################
5193.csv
**************************************************
2022-02-24 17:47:56.691836
2022-02-24 17:48:00.414875
2022-02-24 17:48:18.509463
##################################################
5194.csv
**************************************************
2022-02-24 17:48:24.911335
2022-02-24 17:48:28.427930
2022-02-24 17:48:46.152504
##################################################
5195.csv
**************************************************
2022-02-24 17:48:52.511491
2022-02-24 17:48:56.390114
2022-02-24 17:49:15.871990
########################################

In [26]:
file_path = 'E:\datatest\PseDPC\d5\d520'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
5201.csv
**************************************************
2022-02-24 17:51:23.797725
2022-02-24 17:51:27.488850
2022-02-24 17:51:45.061834
##################################################
5202.csv
**************************************************
2022-02-24 17:51:54.358960
2022-02-24 17:51:58.527807
2022-02-24 17:52:17.230766
##################################################
5203.csv
**************************************************
2022-02-24 17:52:26.727358
2022-02-24 17:52:30.792482
2022-02-24 17:52:49.198237
##################################################
5204.csv
**************************************************
2022-02-24 17:52:58.469432
2022-02-24 17:53:02.138615
2022-02-24 17:53:20.315984
##################################################
5205.csv
**************************************************
2022-02-24 17:53:29.649012
2022-02-24 17:53:33.377037
2022-02-24 17:53:51.467635
########################################